# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/cities.csv"
city_weather_df = pd.read_csv(output_data_file, index_col=0)
city_weather_df


,city,cloudiness,country,date,humidity,lat,lng,max_temp,wind_speed
0,Atuona,0,PF,1584327400,65,-9.80,-139.03,82.20,8.75
1,Castro,13,BR,1584326936,86,-24.79,-50.01,65.79,3.42
2,Yellowknife,20,CA,1584327250,66,62.46,-114.35,10.40,18.34
3,La Ciotat,0,FR,1584327401,57,43.17,5.60,55.40,10.29
4,Rikitea,3,PF,1584327401,79,-23.12,-134.97,79.63,12.71
...,...,...,...,...,...,...,...,...,...
523,Abu Samrah,2,SY,1584327428,86,35.30,37.18,51.85,4.85
524,Soe,42,ID,1584327060,58,-9.86,124.28,85.14,4.56
525,Kattivākkam,40,IN,1584326876,83,13.22,80.32,84.00,2.24
526,Kathmandu,20,NP,1584327386,62,27.72,85.32,55.40,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# gmaps
gmaps.configure(api_key=g_key)


In [4]:
# Store lat and long in locations
locations = city_weather_df[['lat','lng']]


In [5]:
fig = gmaps.figure()

# Humidity
heat_layer = gmaps.heatmap_layer(locations, weights=city_weather_df['humidity'], 
                                 dissipating=False, max_intensity= 10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create variables for fitting weather criteria
temp = city_weather_df['max_temp']
wind_speed = city_weather_df['wind_speed']
cloudiness = city_weather_df['cloudiness']

# Create chart and set values
weather_criteria_df = city_weather_df.loc[(temp <80)& 
                                          (wind_speed<10)&
                                          (cloudiness<5)]

# Make sure there are no null values
weather_criteria_df.count()

city          75
cloudiness    75
country       73
date          75
humidity      75
lat           75
lng           75
max_temp      75
wind_speed    75
dtype: int64

In [7]:
weather_criteria_df

,city,cloudiness,country,date,humidity,lat,lng,max_temp,wind_speed
7,Kumul,0,CN,1584327401,18,42.80,93.45,47.59,3.09
8,Gobabis,0,NaN,1584327107,88,-22.45,18.97,69.80,3.06
17,Sobolevo,0,RU,1584327401,69,54.43,31.90,26.82,7.07
30,Nantucket,1,US,1584327005,74,41.28,-70.10,30.99,3.36
42,Mount Gambier,0,AU,1584327059,43,-37.83,140.77,69.80,5.82
...,...,...,...,...,...,...,...,...,...
483,Aksay,0,RU,1584327426,73,47.26,39.87,19.40,6.71
488,Ryde,0,GB,1584327426,93,50.73,-1.16,42.01,3.36
507,Songjianghe,0,CN,1584327427,52,42.19,127.48,27.12,8.95
510,Fort Myers Beach,1,US,1584327278,94,26.45,-81.95,75.20,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame(weather_criteria_df)
hotel_df

,city,cloudiness,country,date,humidity,lat,lng,max_temp,wind_speed
7,Kumul,0,CN,1584327401,18,42.80,93.45,47.59,3.09
8,Gobabis,0,NaN,1584327107,88,-22.45,18.97,69.80,3.06
17,Sobolevo,0,RU,1584327401,69,54.43,31.90,26.82,7.07
30,Nantucket,1,US,1584327005,74,41.28,-70.10,30.99,3.36
42,Mount Gambier,0,AU,1584327059,43,-37.83,140.77,69.80,5.82
...,...,...,...,...,...,...,...,...,...
483,Aksay,0,RU,1584327426,73,47.26,39.87,19.40,6.71
488,Ryde,0,GB,1584327426,93,50.73,-1.16,42.01,3.36
507,Songjianghe,0,CN,1584327427,52,42.19,127.48,27.12,8.95
510,Fort Myers Beach,1,US,1584327278,94,26.45,-81.95,75.20,4.70


In [9]:
hotel_name = []


params = {'key': g_key,
          'radius': 5000,
          'query':'hotel'
         }


for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    lat = row['lat']
    lng = row['lng']
    params['location'] = f'{lat},{lng}'

    
    
    response = requests.get(base_url, params=params).json()
  
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")


hotel_df


,city,cloudiness,country,date,humidity,lat,lng,max_temp,wind_speed,Hotel Name
7,Kumul,0,CN,1584327401,18,42.80,93.45,47.59,3.09,7 Days Inn
8,Gobabis,0,NaN,1584327107,88,-22.45,18.97,69.80,3.06,Xain Quaz Camp
17,Sobolevo,0,RU,1584327401,69,54.43,31.90,26.82,7.07,Tourist baza Logi
30,Nantucket,1,US,1584327005,74,41.28,-70.10,30.99,3.36,The Nantucket Hotel & Resort
42,Mount Gambier,0,AU,1584327059,43,-37.83,140.77,69.80,5.82,Federal Hotel
...,...,...,...,...,...,...,...,...,...,...
483,Aksay,0,RU,1584327426,73,47.26,39.87,19.40,6.71,Legenda
488,Ryde,0,GB,1584327426,93,50.73,-1.16,42.01,3.36,Best Western Royal Beach Hotel
507,Songjianghe,0,CN,1584327427,52,42.19,127.48,27.12,8.95,Sheraton Changbaishan Resort
510,Fort Myers Beach,1,US,1584327278,94,26.45,-81.95,75.20,4.70,Outrigger Beach Resort


In [10]:
hotel_df

,city,cloudiness,country,date,humidity,lat,lng,max_temp,wind_speed,Hotel Name
7,Kumul,0,CN,1584327401,18,42.80,93.45,47.59,3.09,7 Days Inn
8,Gobabis,0,NaN,1584327107,88,-22.45,18.97,69.80,3.06,Xain Quaz Camp
17,Sobolevo,0,RU,1584327401,69,54.43,31.90,26.82,7.07,Tourist baza Logi
30,Nantucket,1,US,1584327005,74,41.28,-70.10,30.99,3.36,The Nantucket Hotel & Resort
42,Mount Gambier,0,AU,1584327059,43,-37.83,140.77,69.80,5.82,Federal Hotel
...,...,...,...,...,...,...,...,...,...,...
483,Aksay,0,RU,1584327426,73,47.26,39.87,19.40,6.71,Legenda
488,Ryde,0,GB,1584327426,93,50.73,-1.16,42.01,3.36,Best Western Royal Beach Hotel
507,Songjianghe,0,CN,1584327427,52,42.19,127.48,27.12,8.95,Sheraton Changbaishan Resort
510,Fort Myers Beach,1,US,1584327278,94,26.45,-81.95,75.20,4.70,Outrigger Beach Resort


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [12]:
# Add marker layer ontop of heat map

arkers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)


# Display Map
fig

NameError: name 'markers' is not defined